Dieser Quellcode ist Bestandteil der Dissertation von Ines Reinecke
vorgelegt am 11.07.2023 der Technischen Universität Dresden, Medizinische Fakultät

Dieser Code enthält 

1. laden des Datensatzes DS-Med (nach Durchführung der Maßnahmen zur Verbesserung der Datenstuktur) bereits im OMOP Format (Eingangsgröße hier das Ergebnis von Script 01_verbesserte-DS-Med-to-omop) 
2. Laden der ATC nach RxNorm Mappings
3. Zusammenführen der Medikationsverordnungen mit den Mappings
4. Speichern der Medikationsverordnungen, verbessert und mit den concept_ids in der Spalte drug_concept_id mit RxNorm Standard-Terminologie, wenn möglich - im OMOP Format

In [23]:
import pandas as pd
import numpy as np
import os
import datetime
from fuzzywuzzy import fuzz, process
import re
from typing import List
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3, venn3_circles
from pandas_profiling import ProfileReport
# remove later
import warnings
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text
warnings.filterwarnings('ignore')

In [ ]:

# Lesen der Medikationsverordnungen - bereits verbessert und im OMOP Format der Tabelle drug_exposure 
medication_orders = pd.read_csv('../data_results/02_data_to_omop+terminology_results/Schritt-2-DQD-DS-Med.csv', skipinitialspace=True, low_memory=False, lineterminator='\n').fillna(str())


In [ ]:

#  Lesen aller Mappings von ATC nach RxNorm mit den Beziehungstypen ATC - RxNorm pr lat, ATC - RxNorm pr up, ATC - RxNorm sec lat, ATC - RxNorm sec up, Maps to   
atc_rxnorm_relationship = pd.read_csv('../data_in/ATC-RxNorm-Rel.csv', sep='\t')

# Reduzierung der Mappings auf den Beziehungstypen "Maps To"
atc_rxnorm_relationship_mapsTo = atc_rxnorm_relationship.loc[atc_rxnorm_relationship['rel_id']=='Maps to']

In [6]:

# Gruppierung der Mappings vom Typ "Maps to" nach der Anzahl der Mappings pro ATC Code 
atc_rxnorm_relationship_mapsTo_grouped = atc_rxnorm_relationship_mapsTo.groupby(['concept_id_atc', 'rel_id']).size().reset_index(name='count')


In [7]:
print ("Anzahl von ATC codes mit exakt 1 Mapping nach RxNorm",len(atc_rxnorm_relationship_mapsTo_grouped.loc[atc_rxnorm_relationship_mapsTo_grouped['count']==1]))
print ("Anzahl von ATC codes mit exakt 2 Mappings nach RxNorm",len(atc_rxnorm_relationship_mapsTo_grouped.loc[atc_rxnorm_relationship_mapsTo_grouped['count']==2]))
print ("Anzahl von ATC codes mit exakt 3 Mappings nach RxNorm",len(atc_rxnorm_relationship_mapsTo_grouped.loc[atc_rxnorm_relationship_mapsTo_grouped['count']==3]))
print ("Anzahl von ATC codes mehr als 3 Mappings nach RxNorm",len(atc_rxnorm_relationship_mapsTo_grouped.loc[atc_rxnorm_relationship_mapsTo_grouped['count']>3]))

Anzahl von ATC codes mit exakt 1 Mapping nach RxNorm 4285
Anzahl von ATC codes mit exakt 2 Mappings nach RxNorm 290
Anzahl von ATC codes mit exakt 3 Mappings nach RxNorm 77
Anzahl von ATC codes mehr als 3 Mappings nach RxNorm 26


In [8]:
# Reduzierung des Dataframes mit den Mappings von ATC nach RxNorm auf die zwei Spalten mit den concept_ids
# in dem Dataframe können die ATC Codes mehrfach vorkommen - sie haben dann mehr als ein explizites Mapping nach RxNorm 

atc_rxnorm_relationship_mapsTo = atc_rxnorm_relationship_mapsTo[['concept_id_atc','concept_id_rx']]

In [9]:

# Neues Dataframe, welches die Anzahl der expliziten Mappings von ATC Codes nach RxNorm enthält
atc_rxnorm_relationship_mapsTo_grouped = atc_rxnorm_relationship_mapsTo_grouped[['concept_id_atc','count']]


In [10]:
# Zusammenführunge der Medikationsverordnungen im OMOP Format mit den Mappings nach RxNorm, aber nur die gruppierte Variante, um zu prüfen, wieviele Medikationsverordnungen einen ATC Code haben, der zu mehr als einem RxNorm Code explizit gemappt wird

rxnorm_medication_orders_conform_check_number = pd.merge(medication_orders, atc_rxnorm_relationship_mapsTo_grouped, left_on="drug_source_concept_id", right_on="concept_id_atc", how="left")

In [13]:

# Auffüllen des generierten Dataframes mit 0 Werten, wenn keine Anzahl an Mappings existiert, also alle ATC Codes in den Medikationsverordnungen ohne explizites Mapping nach RxNorm
rxnorm_medication_orders_conform_check_number["count"] = rxnorm_medication_orders_conform_check_number["count"].fillna(0)


# Änderung des Datentyps der Spalte "count" nach Integer                                                              
rxnorm_medication_orders_conform_check_number['count'] = rxnorm_medication_orders_conform_check_number['count'].astype('Int64')

In [14]:
print("Number of medication orders without mapping to RxNorm: ",len(rxnorm_medication_orders_conform_check_number.loc[rxnorm_medication_orders_conform_check_number["count"]==0]))
print("Number of medication orders with mapping to RxNorm: ",len(rxnorm_medication_orders_conform_check_number.loc[rxnorm_medication_orders_conform_check_number["count"]==1]))
print("Number of medication orders with mapping to RxNorm: ",len(rxnorm_medication_orders_conform_check_number.loc[rxnorm_medication_orders_conform_check_number["count"]==2]))
print("Number of medication orders with mapping to RxNorm: ",len(rxnorm_medication_orders_conform_check_number.loc[rxnorm_medication_orders_conform_check_number["count"]==3]))
print("Number of medication orders with mapping to RxNorm: ",len(rxnorm_medication_orders_conform_check_number.loc[rxnorm_medication_orders_conform_check_number["count"]>3]))
print("Number of medication orders with at least one mapping to RxNorm via Maps to: ",len(rxnorm_medication_orders_conform_check_number.loc[rxnorm_medication_orders_conform_check_number["count"]>0]))

Number of medication orders without mapping to RxNorm:  598823
Number of medication orders with mapping to RxNorm:  1155692
Number of medication orders with mapping to RxNorm:  13632
Number of medication orders with mapping to RxNorm:  6
Number of medication orders with mapping to RxNorm:  0
Number of medication orders with at least one mapping to RxNorm via Maps to:  1169330


In [16]:

# Die Medikamentenverordnungen werden mit den RxNorm-Zuordnungen aus dem Vokabular zusammengeführt. 
# Dadurch wird das ursprüngliche DataFrame der Medikamentenverordnungen um zusätzliche Zeilen erweitert, abhängig von der Anzahl der zusätzlichen Zuordnungen. 
# Es werden sowohl 1-zu-2- als auch 1-zu-3-Zuordungen von ATC zu RxNorm berücksichtigt.

rxnorm_medication_orders_omop_rxnorm = pd.merge(medication_orders, atc_rxnorm_relationship_mapsTo, left_on="drug_source_concept_id", right_on="concept_id_atc", how="left")

In [17]:

# Auffüllen aller Nan Werte mit 0
rxnorm_medication_orders_omop_rxnorm[["concept_id_atc","concept_id_rx"]] = rxnorm_medication_orders_omop_rxnorm[["concept_id_atc","concept_id_rx"]].fillna(0)

In [18]:

# Überführen des Wertes für die durg_concept_id aus der Spalte concept_id_rx im Falle diese Spalte ist nicht leer
rxnorm_medication_orders_omop_rxnorm['drug_concept_id'] = np.where(
    rxnorm_medication_orders_omop_rxnorm['concept_id_rx'].notnull(),
    rxnorm_medication_orders_omop_rxnorm['concept_id_rx'],
    rxnorm_medication_orders_omop_rxnorm['drug_concept_id']
)

# Entfernen der überflüssigen Spalten aus dem Dataframe der Medikationsverordnugnen nach der Zusammenführung mit den RxNorm Codes
rxnorm_medication_orders_omop_rxnorm=rxnorm_medication_orders_omop_rxnorm.drop(['concept_id_rx','concept_id_atc'], axis=1)

In [19]:

print("Neue Anzahl an Medikationsverordnungen in der Tabelle drug_exposure (durch die Vervielfältigung aufgrund von multi Mappings von ATC nach RxNorm): ",len(rxnorm_medication_orders_omop_rxnorm))


Neue Anzahl an Medikationsverordnungen in der Tabelle drug_exposure (durch die Vervielfältigung aufgrund von multi Mappings von ATC nach RxNorm):  1781797


In [20]:

# Aufgrund der Duplikation bestimmter Einträge von Medikationsverordnungen, bei expliziten MultiMappings nach RxNorm, muss die eindeutige ID in der OMOP Tabelle drug_exposure wieder mit eindeutigen IDs generiert werden
# erfolgt hier auf Basis des Index des Dataframes

rxnorm_medication_orders_omop_rxnorm['drug_exposure_id'] = rxnorm_medication_orders_omop_rxnorm.index


In [21]:

# Änderung eines Datentyps der Spalte "drug_concept_id" im Dataframe der OMOP konformen Medikationsverordnungen                                                              
rxnorm_medication_orders_omop_rxnorm['drug_concept_id'] = rxnorm_medication_orders_omop_rxnorm['drug_concept_id'].astype('Int64')

In [22]:

# Speichern der Medikationsverordnungen für die OMOP Tabelle drug_exposure
rxnorm_medication_orders_omop_rxnorm.to_csv('../data_results/02_data_to_omop+terminology_results/Schritt-3-DQD-DS-Med.csv', index=False)